In [27]:
import cv2
import numpy as np

In [28]:
# Cargar la red YOLO preentrenada
net = cv2.dnn.readNet('./../../yolov3.weights', './../../yolov3.cfg')
layer_names = net.getUnconnectedOutLayersNames()

cap = cv2.VideoCapture(0)

In [29]:


while True:
    ret, frame = cap.read()

    # Obtener las dimensiones de la imagen
    height, width, _ = frame.shape

    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Obtener las detecciones
    detections = net.forward(layer_names)

    # Procesar las detecciones
    boxes = []
    confidences = []

    for detection in detections[0]:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > 0.5 and class_id == 2:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Coordenadas de la esquina superior izquierda
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))

    # Aplicar NMS
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Dibujar rectángulo solo para las detecciones después de NMS
    for i in indices:
        x, y, w, h = boxes[i]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

    # Mostrar la imagen resultante
    cv2.imshow('Car Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [30]:
#PROCESAMIENTO DE MATRICULAS